In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 4.7MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [4]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/dw_matrix"


/content/drive/My Drive/Colab Notebooks/dw_matrix/dw_matrix


In [5]:
ls

data/       day4_meta.ipynb  dw_matrix/         LICENSE      README.md
day4.ipynb  day5.ipynb       HelloGitHub.ipynb  matrix_one/


In [6]:
df = pd.read_csv('data/men_shoes.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df = pd.read_csv('data/men_shoes.csv',low_memory=False)

In [8]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [14]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [15]:
model = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [25]:
df.features.head().values


array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [27]:
test = {'key' : 'values'}
test['key']
str(test)


"{'key': 'values'}"

In [29]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
      key = item['key'].lower().strip()
      value = item['value'][0].lower().strip()

      output_dict[key] = value
  return output_dict


df['features_parsed'] = df['features'].map(parse_features)

In [47]:
keys = set()
df['features_parsed'].map( lambda x: keys.update(x.keys()) )
len(keys)

476

In [49]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
    df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)


In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False== df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [56]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]
df['feat_metal type_cat'] = df['feat_metal type'].factorize()[0]
df['feat_shape_cat'] = df['feat_shape'].factorize()[0]

In [64]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [0]:
feats = ['']


In [0]:
feats = ['brand_cat','feat_brand_cat','feat_metal type_cat','feat_shape_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_sport_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats,model)

In [95]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.2492847702805, 4.261844991692812)


Weight,Feature
0.2562 ± 0.0077,brand_cat
0.1035 ± 0.0075,feat_material_cat
0.0253 ± 0.0026,feat_gender_cat
0.0178 ± 0.0008,feat_brand_cat
0.0126 ± 0.0006,feat_shape_cat
0.0089 ± 0.0012,feat_metal type_cat
0.0031 ± 0.0011,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [85]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'sport': 'skate shoes', 'occasion': 'skate shoes', 'material': 'leather', 'gender': 'men', 'shoe category': "men's shoes", 'fabric content': 'leather', 'color': 'blue spark/white/black', 'model': '819674 410', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'skateboarding shoes', 'manufacturer part number': '819674 410', 'brand': 'nike', 'shoe width': 'd(m)'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'dark pewter/black/grey/volt', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '631754007', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {},
       {'season': 'all-season', 'material': 'synthetic', 'gender': 'men', 'shoe size': '7.5', 'size': '7.5', 'color': 'black', 'model': '653664 003', 'manufacturer part number': '653664 003', 'brand': 'nike', 'age group': 'adult'},
       {'gender': 'men', 'clothing size': '9', 'brand': 'nike', 'color': 'black'}],
      dtype=object)

In [0]:
!git add day5.ipynb

In [97]:
!git commit -m "improved price predition"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@89269b63099d.(none)')


In [0]:
!git config --global user.email "annatylenda@op.pl"
!git config --global user.name "anna"

In [99]:
!git push -u origin master

fatal: could not read Password for 'https://b892564d2e188dea4efaa5642620b4278a518e9c@github.com': No such device or address


In [0]:
GITHUB_TOKEN = ''

In [0]:

GITHUB_URL = 'https://'.format(GITHUB_TOKEN)

In [103]:
GITHUB_URL

'https://.git'

In [0]:
!git clone {GITHUB_URL}